In [1]:
import pandas as pd

In [2]:
pd.set_option("display.max_columns", 500)

## Datasets

- `npidata_pfile`: Data File - File contains the FOIA-disclosable NPPES provider data
- `othername_pfile`: Other Name Reference File - File contains additional Other Names associated with Type 2 NPIs
- `pl_pfile`: Practice Location Reference File - File contains all of the non-primary Practice Locations associated with Type 1 and Type 2 NPIs
- `endpoint_pfile`: Endpoint Reference File - File contains all Endpoints associated with Type 1 and Type 2 NPIs

## Subsetting `npidata`

Read-in the data from file

In [3]:
npidata_raw = pd.read_csv("../data/npidata_pfile_20050523-20210207.csv", nrows=1000)
npidata_raw.head()

,NPI,Entity Type Code,Replacement NPI,Employer Identification Number (EIN),Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider Other Organization Name,Provider Other Organization Name Type Code,Provider Other Last Name,Provider Other First Name,Provider Other Middle Name,Provider Other Name Prefix Text,Provider Other Name Suffix Text,Provider Other Credential Text,Provider Other Last Name Type Code,Provider First Line Business Mailing Address,Provider Second Line Business Mailing Address,Provider Business Mailing Address City Name,Provider Business Mailing Address State Name,Provider Business Mailing Address Postal Code,Provider Business Mailing Address Country Code (If outside U.S.),Provider Business Mailing Address Telephone Number,Provider Business Mailing Address Fax Number,Provider First Line Business Practice Location Address,Provider Second Line Business Practice Location Address,Provider Business Practice Location Address City Name,Provider Business Practice Location Address State Name,Provider Business Practice Location Address Postal Code,Provider Business Practice Location Address Country Code (If outside U.S.),Provider Business Practice Location Address Telephone Number,Provider Business Practice Location Address Fax Number,Provider Enumeration Date,Last Update Date,NPI Deactivation Reason Code,NPI Deactivation Date,NPI Reactivation Date,Provider Gender Code,Authorized Official Last Name,Authorized Official First Name,Authorized Official Middle Name,Authorized Official Title or Position,Authorized Official Telephone Number,Healthcare Provider Taxonomy Code_1,Provider License Number_1,Provider License Number State Code_1,Healthcare Provider Primary Taxonomy Switch_1,Healthcare Provider Taxonomy Code_2,Provider License Number_2,Provider License Number State Code_2,Healthcare Provider Primary Taxonomy Switch_2,Healthcare Provider Taxonomy Code_3,Provider License Number_3,Provider License Number State Code_3,Healthcare Provider Primary Taxonomy Switch_3,Healthcare Provider Taxonomy Code_4,Provider License Number_4,Provider License Number State Code_4,Healthcare Provider Primary Taxonomy Switch_4,Healthcare Provider Taxonomy Code_5,Provider License Number_5,Provider License Number State Code_5,Healthcare Provider Primary Taxonomy Switch_5,Healthcare Provider Taxonomy Code_6,Provider License Number_6,Provider License Number State Code_6,Healthcare Provider Primary Taxonomy Switch_6,Healthcare Provider Taxonomy Code_7,Provider License Number_7,Provider License Number State Code_7,Healthcare Provider Primary Taxonomy Switch_7,Healthcare Provider Taxonomy Code_8,Provider License Number_8,Provider License Number State Code_8,Healthcare Provider Primary Taxonomy Switch_8,Healthcare Provider Taxonomy Code_9,Provider License Number_9,Provider License Number State Code_9,Healthcare Provider Primary Taxonomy Switch_9,Healthcare Provider Taxonomy Code_10,Provider License Number_10,Provider License Number State Code_10,Healthcare Provider Primary Taxonomy Switch_10,Healthcare Provider Taxonomy Code_11,Provider License Number_11,Provider License Number State Code_11,Healthcare Provider Primary Taxonomy Switch_11,Healthcare Provider Taxonomy Code_12,Provider License Number_12,Provider License Number State Code_12,Healthcare Provider Primary Taxonomy Switch_12,Healthcare Provider Taxonomy Code_13,Provider License Number_13,Provider License Number State Code_13,Healthcare Provider Primary Taxonomy Switch_13,Healthcare Provider Taxonomy Code_14,Provider License Number_14,Provider License Number State Code_14,Healthcare Provider Primary Taxonomy Switch_14,Healthcare Provider Taxonomy Code_15,Provider License Number_15,Provider License Number State Code_15,Healthcare Provider Primary Taxonomy Switch_15,Other Provider Identifier_1,Other Provider Identifier Type Code_1,Other Provider Ide

Only keep the needed columns

In [4]:
npidata = pd.concat([
    npidata_raw[['NPI']],
    # Entity Type Code: 1 = Provider (doctors, nurses, etc.) / 2 = Facility (Hospitals, Urgent Care, Doctors Offices) 
    npidata_raw[['Entity Type Code']],
    # Entity Name: Either First/Last or Organization or Other Organization Name contained in the following fields:
    npidata_raw.loc[:, 'Provider Organization Name (Legal Business Name)':'Provider Credential Text'],
    # Address: Business Practice Location (not mailing), contained in the following fields:
    npidata_raw.loc[:, 'Provider First Line Business Mailing Address':'Provider Business Mailing Address Postal Code'],
], axis=1)

npidata.head()

,NPI,Entity Type Code,Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider First Line Business Mailing Address,Provider Second Line Business Mailing Address,Provider Business Mailing Address City Name,Provider Business Mailing Address State Name,Provider Business Mailing Address Postal Code
0,1679576722,1.0,NaN,WIEBE,DAVID,A,NaN,NaN,M.D.,PO BOX 2168,NaN,KEARNEY,NE,688482168.0
1,1588667638,1.0,NaN,PILCHER,WILLIAM,C,DR.,NaN,MD,1824 KING STREET,SUITE 300,JACKSONVILLE,FL,322044736.0
2,1497758544,2.0,"CUMBERLAND COUNTY HOSPITAL SYSTEM, INC",NaN,NaN,NaN,NaN,NaN,NaN,3418 VILLAGE DR,NaN,FAYETTEVILLE,NC,283044552.0
3,1306849450,1.0,NaN,SMITSON,HAROLD,LEROY,DR.,II,M.D.,810 LUCAS DR,NaN,ATHENS,TX,757513446.0
4,1215930367,1.0,NaN,GRESSOT,LAURENT,NaN,DR.,NaN,M.D.,17323 RED OAK DR,NaN,HOUSTON,TX,770901243.0


Check the current shape of the dataframe

In [5]:
npidata.shape

(1000, 14)

**We will handle the Primary Taxonomy separately and left-join back to `npidata` later**

## Handling The Primary Taxonomy


A provider can have up to 15 taxonomy codes, but we want the one which has Primary Switch = Y in the associated 'Healthcare Provider Primary Taxonomy Switch*' field. Note that this does not always occur in spot 1.

In [6]:
npi_taxonomy = pd.concat([
    npidata_raw[['NPI']],
    # The provider's taxonomy code, which is contained in one of the 'Healthcare Provider Taxonomy Code*' columns
    npidata_raw[npidata_raw.columns[pd.Series(npidata_raw.columns).str.startswith('Healthcare Provider Taxonomy Code_')]],
    npidata_raw[npidata_raw.columns[pd.Series(npidata_raw.columns).str.startswith('Healthcare Provider Primary Taxonomy Switch_')]]
], axis=1)

npi_taxonomy.head()

,NPI,Healthcare Provider Taxonomy Code_1,Healthcare Provider Taxonomy Code_2,Healthcare Provider Taxonomy Code_3,Healthcare Provider Taxonomy Code_4,Healthcare Provider Taxonomy Code_5,Healthcare Provider Taxonomy Code_6,Healthcare Provider Taxonomy Code_7,Healthcare Provider Taxonomy Code_8,Healthcare Provider Taxonomy Code_9,Healthcare Provider Taxonomy Code_10,Healthcare Provider Taxonomy Code_11,Healthcare Provider Taxonomy Code_12,Healthcare Provider Taxonomy Code_13,Healthcare Provider Taxonomy Code_14,Healthcare Provider Taxonomy Code_15,Healthcare Provider Primary Taxonomy Switch_1,Healthcare Provider Primary Taxonomy Switch_2,Healthcare Provider Primary Taxonomy Switch_3,Healthcare Provider Primary Taxonomy Switch_4,Healthcare Provider Primary Taxonomy Switch_5,Healthcare Provider Primary Taxonomy Switch_6,Healthcare Provider Primary Taxonomy Switch_7,Healthcare Provider Primary Taxonomy Switch_8,Healthcare Provider Primary Taxonomy Switch_9,Healthcare Provider Primary Taxonomy Switch_10,Healthcare Provider Primary Taxonomy Switch_11,Healthcare Provider Primary Taxonomy Switch_12,Healthcare Provider Primary Taxonomy Switch_13,Healthcare Provider Primary Taxonomy Switch_14,Healthcare Provider Primary Taxonomy Switch_15
0,1679576722,207X00000X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1588667638,207RC0000X,207RC0000X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1497758544,251G00000X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1306849450,2085R0202X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1215930367,174400000X,207RH0003X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Apply `wide_to_long` to get the matrix of taxonomy in a long format and filter for only the primary taxonomies (**warning: This would get rid of NPI's without primary taxonomies. Do a left-join on `npidata` to fix this**)

In [7]:
# Pivot from widet to long format
npi_taxonomy = pd.wide_to_long(
    npi_taxonomy,
    stubnames=['Healthcare Provider Taxonomy Code', 'Healthcare Provider Primary Taxonomy Switch'],
    i=['NPI'], 
    j='primary_taxonomy_index',
    sep="_"
)

# Only keep the primary taxonomy
npi_taxonomy = npi_taxonomy[npi_taxonomy['Healthcare Provider Primary Taxonomy Switch'] == 'Y']

# Housekeeping
npi_taxonomy = npi_taxonomy.reset_index()\
    .drop(columns=['primary_taxonomy_index', 'Healthcare Provider Primary Taxonomy Switch'])\
    .rename({ 'Healthcare Provider Taxonomy Code': 'primary_taxonomy' }, axis=1)

npi_taxonomy

,NPI,primary_taxonomy
0,1679576722,207X00000X
1,1497758544,251G00000X
2,1306849450,2085R0202X
3,1023011178,251G00000X
4,1932102084,207RC0000X
...,...,...
932,1699778829,207X00000X
933,1811990047,333600000X
934,1619970845,3336H0001X
935,1164425336,3336H0001X


Finally, merge back `npi_taxonomy` to `npidata`. Account for npis without `primary_taxonomy` by doing a left-join

## Merging `npidata` and `npi_taxonomy`

In [8]:
npidata = npidata.merge(
    npi_taxonomy,
    how='left',
    on='NPI'
)

npidata.head()

,NPI,Entity Type Code,Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider First Line Business Mailing Address,Provider Second Line Business Mailing Address,Provider Business Mailing Address City Name,Provider Business Mailing Address State Name,Provider Business Mailing Address Postal Code,primary_taxonomy
0,1679576722,1.0,NaN,WIEBE,DAVID,A,NaN,NaN,M.D.,PO BOX 2168,NaN,KEARNEY,NE,688482168.0,207X00000X
1,1588667638,1.0,NaN,PILCHER,WILLIAM,C,DR.,NaN,MD,1824 KING STREET,SUITE 300,JACKSONVILLE,FL,322044736.0,207RC0000X
2,1497758544,2.0,"CUMBERLAND COUNTY HOSPITAL SYSTEM, INC",NaN,NaN,NaN,NaN,NaN,NaN,3418 VILLAGE DR,NaN,FAYETTEVILLE,NC,283044552.0,251G00000X
3,1306849450,1.0,NaN,SMITSON,HAROLD,LEROY,DR.,II,M.D.,810 LUCAS DR,NaN,ATHENS,TX,757513446.0,2085R0202X
4,1215930367,1.0,NaN,GRESSOT,LAURENT,NaN,DR.,NaN,M.D.,17323 RED OAK DR,NaN,HOUSTON,TX,770901243.0,207RH0003X


## Renaming Columns

In [9]:
npidata = npidata.reset_index().rename({
    'NPI': 'npi',
    'Entity Type Code': 'entity_type_code',
    'Provider Organization Name (Legal Business Name)': 'provider_org_name',
    'Provider Last Name (Legal Name)': 'provider_last_name',
    'Provider First Name': 'provider_first_name',
    'Provider Middle Name': 'provider_middle_name',
    'Provider Name Prefix Text': 'provider_name_prefix',
    'Provider Name Suffix Text': 'provider_name_suffix',
    'Provider Credential Text': 'provider_credential',
    'Provider First Line Business Mailing Address': 'provider_business_address_1',
    'Provider Second Line Business Mailing Address': 'provider_business_address_2',
    'Provider Business Mailing Address City Name': 'provider_business_city',
    'Provider Business Mailing Address State Name': 'provider_business_state',
    'Provider Business Mailing Address Postal Code': 'provider_business_zip'
}, axis=1)

npidata.head()

,index,npi,entity_type_code,provider_org_name,provider_last_name,provider_first_name,provider_middle_name,provider_name_prefix,provider_name_suffix,provider_credential,provider_business_address_1,provider_business_address_2,provider_business_city,provider_business_state,provider_business_zip,primary_taxonomy
0,0,1679576722,1.0,NaN,WIEBE,DAVID,A,NaN,NaN,M.D.,PO BOX 2168,NaN,KEARNEY,NE,688482168.0,207X00000X
1,1,1588667638,1.0,NaN,PILCHER,WILLIAM,C,DR.,NaN,MD,1824 KING STREET,SUITE 300,JACKSONVILLE,FL,322044736.0,207RC0000X
2,2,1497758544,2.0,"CUMBERLAND COUNTY HOSPITAL SYSTEM, INC",NaN,NaN,NaN,NaN,NaN,NaN,3418 VILLAGE DR,NaN,FAYETTEVILLE,NC,283044552.0,251G00000X
3,3,1306849450,1.0,NaN,SMITSON,HAROLD,LEROY,DR.,II,M.D.,810 LUCAS DR,NaN,ATHENS,TX,757513446.0,2085R0202X
4,4,1215930367,1.0,NaN,GRESSOT,LAURENT,NaN,DR.,NaN,M.D.,17323 RED OAK DR,NaN,HOUSTON,TX,770901243.0,207RH0003X
